In [25]:
import numpy as np

def first_x_sec(seconds, filepath, timecol):
    ''' param: seconds, int, filter out the first seconds of the game
        param: filepath, string, the filepath of the csv datafile
        param: timecol, int, the index of the column that contains time
    '''
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.split(',') for line in lines ]
    if seconds is None:
        return np.array(lines[1:])
    else:
        return np.array(list(filter(lambda line: int(line[timecol])<=seconds, lines[1:])))

# First 5 min: 5*60=300, col 2 is time first line is header
#ability_upgrade = first_x_sec(300, 'dataset/ability_upgrades.csv', 2)
#objectives = first_x_sec(300, 'dataset/objectives.csv', 7)
player_time = first_x_sec(300, 'dataset/player_time.csv', 1)


## Match file contains more than just the label
# column 10 contains the True/False value of Radiant win
matches = first_x_sec(None, 'dataset/match.csv', None)



In [47]:
labels = np.array([int(match[9] == 'True') for match in matches])
print('All the labels')
print(labels)

print('Shape of player time matrix')
print('Contains all attributes of matches, the first minutes')
print(player_time.shape)
#print(player_time[1:10])
player_time = player_time.astype(np.int)

## Separate data matches. Each index in data is one match. One match is a 32*x.
## x is the number of logs. We have data every 60 secods, so for 5 min x=5
games = [np.empty(shape=(32,0), dtype=np.int)]*50000
for x in player_time:
    games[x[0]] = np.concatenate((np.array(games[x[0]]), x.reshape(32,1)), axis=1)
#Print the first match log 
print('The pre prossessed data, just the second match')
print(games[1])


All the labels
[1 0 0 ..., 1 1 0]
Shape of player time matrix
Contains all attributes of matches, the first minutes
(299994, 32)
The pre prossessed data, just the second match
[[   1    1    1    1    1    1]
 [   0   60  120  180  240  300]
 [   0   99  199  299  399  798]
 [   0    0    0    0    0    0]
 [   0   31  288  461  607  910]
 [   0  138  358  744 1146 1479]
 [   0    1    4   11   18   20]
 [   0   93  350  647  939 1348]
 [   0   99  280  441 1018 1118]
 [   0    0    3    8    8    8]
 [   0    0  144  264  493  627]
 [   0  178  326  426  651  950]
 [   0    2    3    3    6   10]
 [   0  124  165  206  495  735]
 [   0  280  542  891 1310 1574]
 [   0    2    6   12   17   21]
 [   0  327  781 1225 1682 2059]
 [   0  599  804 1314 1861 2604]
 [   0    0    3    6   11   16]
 [   0  100  222  426  942 1512]
 [   0  183  325  623  770  911]
 [   0    2    3    8    9   10]
 [   0  175  299  670  979 1253]
 [   0   99  199  299  399  582]
 [   0    0    0    0    0    1]

In [27]:
#game_states = np.hstack([game[1:,:] for game in data])
#print(np.shape(game_states))
#print(game_states[:20])

In [32]:
from keras.layers import Dense, Activation
from keras.models import Sequential
import keras

model = Sequential()

model.add(Dense(units=50, input_dim=30))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.0, nesterov=False),
             metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])


In [50]:
print('Shapes of labels')
print(labels.shape)
from keras.utils.np_utils import to_categorical
labels_binary = to_categorical(labels)
print('example of how the labels look like')
print(labels[:10])
print('Re-structure the labels so that it is two dimensions, with prob value of each team winning')
print(labels_binary.shape)
print('example of how the binary labels look like')
print(labels_binary[:10])

print(games[0])
games = np.array([game[:, 1:] for game in games]).T
print(games[0])
print(games)

# separate into train and test
x_train = games[:,:40000]
y_train = labels_binary[:40000,:]
x_test = games[:,40000:]
y_test = labels_binary[40000:,:]
print(y_test.shape)



Shapes of labels
(50000,)
example of how the labels look like
[1 0 0 0 1 1 1 1 0 0]
Re-structure the labels so that it is two dimensions, with prob value of each team winning
(50000, 2)
example of how the binary labels look like
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
[[   0    0    0    0    0    0]
 [   0   60  120  180  240  300]
 [   0  409  546  683  956 1056]
 [   0    0    0    1    1    1]
 [   0   63  283  314  485  649]
 [   0  142  622  927 1264 1451]
 [   0    1    4    9   11   13]
 [   0  186  645 1202 1583 1810]
 [   0  168  330  430  530  630]
 [   0    0    0    0    0    0]
 [   0  125  376  376  391  504]
 [   0  200  345  644  919 1102]
 [   0    0    1    6   11   15]
 [   0  193  698 1172 1610 1888]
 [   0  194  628  806 1281 1708]
 [   0    1    5    7   10   17]
 [   0  125  374  570 1216 1633]
 [   0  174  354  614 1082 1300]
 [   0    2    4    8    8   11]
 [   0   77  437  829 1318 1901]


ValueError: could not broadcast input array from shape (32,5) into shape (32)

In [ ]:
res = model.fit(x_train.T, y_train, epochs=50, batch_size=1000, validation_split=0.8)

In [ ]:
# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test.T, y_test)
print(loss_and_metrics)